In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, color
from skimage.feature import graycomatrix, graycoprops
from skimage.util import img_as_ubyte
from skimage.exposure import rescale_intensity
from scipy.stats import skew, kurtosis
import pandas as pd
import random

image_sources = [
    data.camera(), data.coins(), data.text(),
    data.chelsea(), data.astronaut(), data.rocket()
]
raw_image = random.choice(image_sources)

gray_image = color.rgb2gray(raw_image) if raw_image.ndim == 3 else raw_image
image = img_as_ubyte(gray_image)

plt.imshow(image, cmap='gray')
plt.title("Randomly Selected Grayscale Image")
plt.axis("off")
plt.show()

In [ ]:
DISTANCES = [1]
# ANGLES = [0, np.pi/4, np.pi/2, 3*np.pi/4]
ANGLES = [0]
BLOCK_SIZE = 0

In [ ]:
def glcm_entropy(glcm):
    eps = np.finfo(float).eps
    entropy = -np.sum(glcm * np.log2(glcm + eps), axis=(0, 1))
    return entropy.flatten()

def extract_glcm_features_from_block(block, distances, angles):
    glcm = graycomatrix(block, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
    entropy_vals = glcm_entropy(glcm).reshape(-1)
    # print(glcm)

    contrast_vals = graycoprops(glcm, 'contrast').reshape(-1)
    correlation_vals = graycoprops(glcm, 'correlation').reshape(-1)
    homogeneity_vals = graycoprops(glcm, 'homogeneity').reshape(-1)
    energy_vals = graycoprops(glcm, 'energy').reshape(-1)

    features = []
    for d_idx, d in enumerate(distances):
        for a_idx, a in enumerate(angles):
            idx = d_idx * len(angles) + a_idx
            features.append({
                'distance': d,
                'angle': a,
                'contrast': contrast_vals[idx],
                'correlation': correlation_vals[idx],
                'homogeneity': homogeneity_vals[idx],
                'energy': energy_vals[idx],
                'entropy': entropy_vals[idx],
                'glcm_image': rescale_intensity(glcm[:, :, d_idx, a_idx], out_range=(0, 255)).astype(np.uint8)
            })
    return features

height, width = image.shape
if BLOCK_SIZE == 0:
    blocks = [image]
else:
    blocks = [
        image[y:y+BLOCK_SIZE, x:x+BLOCK_SIZE]
        for y in range(0, height, BLOCK_SIZE)
        for x in range(0, width, BLOCK_SIZE)
        if y + BLOCK_SIZE <= height and x + BLOCK_SIZE <= width
    ]

print(f"Image size = ({width}x{height})")
print(f"Number of blocks = {len(blocks)}")

features_list = []

for b_idx, block in enumerate(blocks):
    print(f"\nProcessing block {b_idx + 1}/{len(blocks)}...")
    block_features = extract_glcm_features_from_block(block, DISTANCES, ANGLES)
    features_list.extend(block_features)

    for f_idx, feat in enumerate(block_features):
        angle_deg = int(np.degrees(feat['angle']))
        print(f"[{b_idx+1}-{f_idx+1}] d={feat['distance']}, θ={angle_deg}°")
        print(f"     contrast     = {feat['contrast']:.4f}")
        print(f"     correlation  = {feat['correlation']:.4f}")
        print(f"     homogeneity  = {feat['homogeneity']:.4f}")
        print(f"     energy       = {feat['energy']:.4f}")
        print(f"     entropy      = {feat['entropy']:.4f}")

In [ ]:
def summarize_glcm_features_per_frame(features_list):
    df = pd.DataFrame([
        {k: v for k, v in f.items() if k != 'glcm_image'} for f in features_list
    ])

    summary = {}
    for feature in ['contrast', 'correlation', 'homogeneity', 'energy', 'entropy']:
        values = df[feature].values
        summary[f"mean_{feature}"] = np.mean(values)
        summary[f"std_{feature}"] = np.std(values)

    return summary


In [ ]:
summary = summarize_glcm_features_per_frame(features_list)

print("Frame-level GLCM Feature Summary:")
for k, v in summary.items():
    print(f"{k:25s}: {v:.4f}")

In [ ]:
for i, feature in enumerate(features_list):
    plt.imshow(feature['glcm_image'], cmap='hot', interpolation='nearest')
    plt.title(f"d={feature['distance']}, θ={int(np.degrees(feature['angle']))}°", fontsize=7)
    plt.axis("off")
    plt.colorbar()
    plt.tight_layout()
    plt.show()